In [3]:
import requests
from bs4 import BeautifulSoup as BS
import config
import mysql.connector
from mysql.connector import errorcode
from datetime import date, datetime, timedelta
import time

headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
mojo_root = 'https://www.boxofficemojo.com/yearly/'

In [4]:
def connect_to_AWS():
    cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        passwd = config.password
    )
    return cnx

def close_connections():
    cursor.close()
    conn.close()
    
conn = connect_to_AWS()
cursor = conn.cursor()
db_name = 'Movies'

In [42]:
close_connections()

In [5]:
## Function to create new DB 
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)

## Try to use the database we are creating.
## If we can't use it, then that means the database doesn't exist
    ## Thus we create the database and switch over to it.
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        conn.database = db_name
    else:
        print(err)
        exit(1)
        
# Function to create tables from a dictionary of tables
# dictionary will have the name of the table as the key and the query to create that table as the value
def create_table(dict_of_tables):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")
        
def drop_tables():
    cursor.execute("""DROP TABLE mg_junction""")
    cursor.execute("""DROP TABLE movies""")
    cursor.execute("""DROP TABLE genres""")
    conn.commit()

In [89]:
TABLES = {}
TABLES['movies'] = """CREATE TABLE movies (
        movieId INT NOT NULL AUTO_INCREMENT,
        movieTitle varchar(255),
        movieGross INT(10),
        movieBudget INT(10),
        movieRunTime REAL,
        movieDistributor varchar(100),
        movieAgeRating varchar(10),
        movieReleaseDate DATE,
        PRIMARY KEY (movieId),
        UNIQUE KEY(movieTitle)
    ) ENGINE=InnoDB"""

TABLES['genres'] = """CREATE TABLE genres (
        genreId INT NOT NULL AUTO_INCREMENT,
        genre varchar(50),
        PRIMARY KEY (genreId),
        UNIQUE KEY(genre)
) ENGINE=InnoDB"""

TABLES['mg_junction'] = """CREATE TABLE `mg_junction` (
  `movieId` INT NOT NULL,
  `genreId` INT NOT NULL,
  PRIMARY KEY (`movieId`, `genreId`),
  INDEX `movieId_idx` (`movieId` ASC),
  CONSTRAINT `moivieId`
    FOREIGN KEY (`movieId`)
    REFERENCES `Movies`.`movies` (`movieId`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `genreId`
    FOREIGN KEY (`genreId`)
    REFERENCES `Movies`.`genres` (`genreId`)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
    ENGINE=InnoDB
"""

In [52]:
drop_tables()

In [53]:
create_table(TABLES)
cursor.execute("""ALTER TABLE `Movies`.`movies` AUTO_INCREMENT = 101;""")
conn.commit()

Creating table movies: OK
Creating table genres: OK
Creating table mg_junction: OK


In [9]:
def get_soup(url):
    try:
        page = requests.get(url, timeout=5)
    # include code to do status check
        if page.status_code != 200:
            print(page.status_code)
        else:
#             print(page.status_code)
            soup = BS(page.content,'html.parser')
    except requests.ConnectionError as e:
        print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
        print(str(e))
    except requests.Timeout as e:
        print("OOPS!! Timeout Error")
        print(str(e))
    except requests.RequestException as e:
        print("OOPS!! General Error")
        print(str(e))
    except KeyboardInterrupt:
        print("Someone closed the program") 
        
    
    time.sleep(1)
    return soup

In [11]:
soup = get_soup(mojo_root)

200


In [12]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <title>
   Movie Box Office Results by Year, 1980-Present - Box Office Mojo
  </title>
  <meta content="year, yearly, box office, box, office, movie, result, average, total, chart, gross, screen, screens, ticket price, ticket, price, cost, budget, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019" name="keywords"/>
  <meta content="Movie Box Office Results by Year, 1980-Present." name="description"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
 </head>
 <body>
  <iframe frameborder="0" height=

In [35]:
print(get_soup('https://www.boxofficemojo.com/yearly/chart/?yr=2019&p=.htm').prettify())

200
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <title>
   2019 Yearly Box Office Results - Box Office Mojo
  </title>
  <meta content="2019, year, yearly, box, office, result, list, movie, movies, listing, listings, top movies, all time, film" name="keywords"/>
  <meta content="Yearly box office results for 2019." name="description"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
 </head>
 <body>
  <iframe frameborder="0" height="1" id="sis_pixel_sitewide" marginheight="0" marginwidth="0" style="display: none;" width="1">
  </iframe>
  <script>
   setTimeout(function(){
        try{
            //sis3.0 pixel
            var cacheBust = Math.random() * 10000000000000000,
                url_sis3 = '//s.amazon-adsystem.com/

In [115]:
def str_to_int(string):
    if string == 'N/A':
        return 0
    string = string.replace(',','')
    temp = string.split()
    if len(temp) == 1:
        return int(temp[0][1:])
    elif temp[-1] == 'million':
        return int(float(temp[0].strip('$'))*(10**6))
    
def str_to_date(string):
    return datetime.strptime(string,"%B %d, %Y").date()

def str_to_time(string):
    temp = string.replace('min.','hrs.').split('hrs.')
    return float(temp[0]) + float(temp[1])/60
    
    

def get_from_years(start_year,end_year):
    for year in list(range(start_year,end_year+1)):
        for page_num in [1]:
            url = f'https://www.boxofficemojo.com/yearly/chart/?page={page_num}&view=releasedate&view2=domestic&yr={year}&p=.htm'
            get_movies(url)
            
def get_movies(list_url):
    soup = get_soup(list_url)
    movie_links = [soup.select('tr td b font a')[i]['href'] for i in range(4,len(soup.select('tr td b font a'))-4)]
    for link in movie_links:
        url = f'https://www.boxofficemojo.com{link}'
#         print(url)
        soup = get_soup(url)
        movie_details = [soup.select('tr td b')[i].text for i in range(len(soup.select('tr td b')))][1:10]
        movie_details = [movie_details[i] for i in range(len(movie_details)) if not movie_details[i].startswith('Domestic')]
        ch = add_movie(movie_details)
        if ch != 0:
            add_genre(movie_details[4].replace(' / ',' ').split(' '))
            connect_movie_genres(movie_details[0].strip(),movie_details[4].replace(' / ',' ').split(' '))

def add_movie(movie_det):
    try:
        cursor.execute("""INSERT INTO movies (
        movieTitle, movieGross, movieBudget, movieRunTime,
        movieDistributor, movieAgeRating, movieReleaseDate)
        VALUES(%s,%s,%s,%s,%s,%s,%s);""",
                      (movie_det[0].strip(),str_to_int(movie_det[1]),str_to_int(movie_det[7]),round(str_to_time(movie_det[5]),2),
                      movie_det[2],movie_det[6],str_to_date(movie_det[3])))
        conn.commit()
        print(f"ADDING [[{movie_det[0].strip()}]] to the 'movies' table.")
        return movie_det[0].strip()
    except:
#         cursor.execute(f"""UPDATE movies SET movieGross = {str_to_int(movie_det[1])} WHERE movieTitle = {movie_det[0]};""")
        print(f"[[{movie_det[0]}]] already exists in the 'movies' table.")
#         conn.commit()
        return 0

def add_genre(movie_det):
    for gen in movie_det:
        query = f"""SELECT * FROM genres WHERE genre LIKE "{gen}";"""
        cursor.execute(query)
        temp = cursor.fetchall()
#         print(type(temp),temp)
        if len(temp) != 0:
            print(f"[[{gen}]] already exists in the 'genre' table.")
        else:
            query = f"""INSERT INTO genres (genre) VALUES ('{gen}');"""
#             print(query)
            cursor.execute(query)
            conn.commit()
            print(f"ADDING [[{gen}]] to the 'genre' table.")
    return 1

def connect_movie_genres(movie_name,movie_genres):
#     print(movie_name, movie_genres)
#     query = f"""SELECT movieId FROM movies WHERE movieTitle = '{movie_name}';"""
    query = f"""SELECT movieId FROM Movies.movies WHERE movieTitle LIKE "{movie_name}";"""
#     print(query)
    cursor.execute(query)
    mId = cursor.fetchall()
#     print(mId)
    for gen in movie_genres:
        query = f"""SELECT genreId FROM genres WHERE genre LIKE "{gen}";"""
        cursor.execute(query)
        gId = cursor.fetchall()
#         print(gId)
        cursor.execute("""INSERT INTO mg_junction VALUES (%s, %s)""",(mId[0][0],gId[0][0]))
        conn.commit()
    return 0
    

In [116]:
temp = get_from_years(2019,2019)

[[Avengers:Endgame]] already exists in the 'movies' table.
[[Captain Marvel ]] already exists in the 'movies' table.
[[Toy Story 4]] already exists in the 'movies' table.
[[The Lion King (2019)]] already exists in the 'movies' table.
[[Aladdin (2019)]] already exists in the 'movies' table.
[[Spider-Man:Far from Home]] already exists in the 'movies' table.
[[Us]] already exists in the 'movies' table.
[[John Wick:Chapter 3 - Parabellum]] already exists in the 'movies' table.
[[How to Train Your Dragon:The Hidden World]] already exists in the 'movies' table.
[[The Secret Life of Pets 2]] already exists in the 'movies' table.
[[Pokemon Detective Pikachu]] already exists in the 'movies' table.
[[Shazam!]] already exists in the 'movies' table.
[[Dumbo (2019)]] already exists in the 'movies' table.
[[Glass]] already exists in the 'movies' table.
[[Godzilla:King of the Monsters]] already exists in the 'movies' table.
[[The Upside]] already exists in the 'movies' table.
[[The LEGO Movie 2:The S

In [106]:
drop_tables()
create_table(TABLES)

Creating table movies: OK
Creating table genres: OK
Creating table mg_junction: OK


In [243]:
print(string_to_int('$20 million'))
print(string_to_int('$856,576,358'))
print(string_to_date('March 8, 2019'))
print(round(string_to_time('1 hrs. 56 min.'),2))

20000000
856576358
2019-03-08
1.93
